In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

Mounted at /content/drive
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project/spectroconv
     |████████████████████████████████| 348 kB 9.1 MB/s 
     |████████████████████████████████| 132 kB 70.7 MB/s 
     |████████████████████████████████| 6.2 MB 25.4 MB/s 
     |████████████████████████████████| 81 kB 11.0 MB/s 
     |████████████████████████████████| 8.7 MB 49.3 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 138 kB 79.2 MB/s 
     |████████████████████████████████| 65 kB 4.4 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Sat Apr 30 03:25:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from models import wb_experiment

NumExpr defaulting to 2 threads.


In [6]:
n_mels = 128

config = {
  'data': {
    'train': {
        'name': f"nsynth-full-{n_mels}-train"
    },
    'val': {
        'name': f"nsynth-full-{n_mels}-val"
    }
  },
  'group': 'cnn',
  'job_type': 'sweep',
  'model': {
      'id': wb_experiment.ModelType.CNN,
      'name': 'CNN'
  },
}

# wandb's sweep config must be exact, no additional dictionary values are allowed.
wb_sweep_config = {
    'name': 'cnn-sweep',
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'method': 'random',
    'metric': {
      'name': 'f1',
      'goal': 'maximize'
    },
    'parameters': {
      'kernel_size': {
          'values': [3, 5]
      },
      'num_channels': {
          'distribution': 'int_uniform',
          'min': 10,
          'max': 50,
      },
      'num_conv_layers': {
          'values': [1,2]
      },
      'linear_layer_size': {
          'distribution': 'int_uniform',
          'min': 60,
          'max': 120,
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 0.00005,
          'max': 0.005,
      },
      'dropout': {# probability of an element to be zeroed (0 = no dropout)
          'distribution': 'uniform',
          'min': .40,
          'max': .60,
      },
      'l2_reg': {
          'value': 0, # trying out no l2 regularizer
          #'distribution': 'log_uniform_values',
          #'min': 0.0000001,
          #'max': 0.01,
      },
      'optimizer': {
          'value': 'adam'
      },
      'epochs': {
          'value': 2
      },
      'batch_size': {
          'value': 100
          #'distribution': 'q_log_uniform_values',
          #'q': 4,
          #'min': 72,
          #'max': 128,
      },
      'linear_reg': {
          'value': False
      }
    }
}

config['sweep_config'] = wb_sweep_config



In [ ]:
sweeper = wb_experiment.SweepExperiment(wb_config=config, login_key=True)
sweeper.run_sweep(iterations = 40, save_all = False)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ut3l9mjt
Sweep URL: https://wandb.ai/jakeval-colab/spectroconv-debug/sweeps/ut3l9mjt
Starting sweep agent: entity=None, project=None, count=40


wandb: Agent Starting Run: vvvkv5q4 with config:
wandb: 	batch_size: 100
wandb: 	dropout: 0.4381140332986429
wandb: 	epochs: 2
wandb: 	kernel_size: 3
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0007332667639623565
wandb: 	linear_layer_size: 107
wandb: 	linear_reg: False
wandb: 	num_channels: 47
wandb: 	num_conv_layers: 2
wandb: 	optimizer: adam
wandb: Currently logged in as: jakeval-colab (use `wandb login --relogin` to force relogin)


hub://jakeval/nsynth-full-128-train-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train-metadata
hub://jakeval/nsynth-full-128-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train
hub://jakeval/nsynth-full-128-val-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val-metadata
hub://jakeval/nsynth-full-128-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]